<a href="https://colab.research.google.com/github/partizanos/computational_finance/blob/master/TP12_Comp_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Series 12
Introduction to Computational Finance
DIMITRIS PROIOS

## Bonds
### Using the same bond as last week, and assuming the required yield is 10%: A bond with 3 years maturity, a face value of 100$ and coupons of 10% paid semi-annually:

|time [years] | coupon [CHF] 
|---|---| 
| 0.5 | 5
| 1 | 5
| 1.5 | 5
| 2 | 5
|2.5 | 5
| 3 | 5


### What is the present value of the bond ?

**Answer**

Value of bonds can be calculated with this formula:
$B = (\sum c e^{rt}) + P e^{rt}$

By replacing with the data of our table the formula becomes:

$B = 5 e^{-0.1 * 0.5} + 5 e^{-0.1* 1} +5 e^{-0.1 * 1.5} +5 e^{-0.1* 2} +5 e^{-0.1 * 2.5}  + 105 e^{-0.1 * 3}$


In [53]:
import numpy as np

discount = lambda n, rate: np.power(np.e, (-n * rate))

coupon_values = lambda coupon, periods: sum(
    [coupon * discount(n, rate) for n in periods]
)

def total_present_value(face_value, coupon, periods, rate):
    cv = coupon_values(coupon, periods)
    #print("coupon values: ", cv )
    
    last_payment = face_value * discount(periods[-1], rate)
    #print("last_payment: ", last_payment)
    
    total_pv = cv + last_payment 
    
    
    return total_pv 


face_value = 100
coupon = 5
periods = np.linspace(0.5, 3, 6).tolist()
rate = 0.1

pv = total_present_value(face_value, coupon, periods, rate)
print("total_pv: ", pv)

total_pv:  99.35744494713597


### Compute the duration and the convexity of the bond.

**Answer**

- Duration:

$D =\frac{\sum_{i=1}^{n} t_i c_i e^{-yt_i}}{B}  $ 

- Convexity:

C = $\frac{1}{P (1+y^2)} \sum_{t=0}^n  {c_i} \frac {(t^2 +t)}{(1+y)^t}$

P: bond price

t: period

y: yield rate

c: coupon value

Convexity is a measure of the curvature, or the degree of the curve, in the relationship between bond prices and bond yields. So when yield change there is a change in the price of the bond

In [57]:
discount = lambda t, rate: np.power(np.e, (-t * rate))
duration_coupon_values = lambda coupon, periods: sum([t * coupon * discount(t, rate) for t in periods])


def duration(face_value, coupon, periods, rate):
  duration_cv = duration_coupon_values(coupon, periods)
  
  tv = total_present_value(face_value, coupon, periods, rate)
  
  duration = duration_cv / tv
  
  print("duration_coupon_values: ", duration_cv)
  print("present value : ", tv)
  print("duration: ", duration)
  
  return duration
  
#Duration of bond
dur_bond=duration(face_value, coupon, periods, rate)


def convexity(P, y, c, face_value,periods):  
  calc_coupon_impact = lambda t, y: (t+t**2)/((1+y)**t) 
  cv = sum([
      c * calc_coupon_impact(t, y )
      for t in periods
  ])
  
  fv =  face_value * calc_coupon_impact(periods[-1], y)
  all_coupons = cv + fv
  return (1 / (P + y **2) ) * all_coupons


P = total_present_value(face_value, coupon, periods, rate)
y = rate 
c = 5
periods=np.linspace(0.5, 3, 6).tolist()
conv = convexity(P, y, c, face_value,periods)
print("convexity: ", conv)

duration_coupon_values:  42.39216110401766
present value :  99.35744494713597
duration:  0.4266631567123409
convexity:  10.414306271249705


### Estimate (with and without convexity) what would be the effect of a x% parallel shift of the yield curve when x equals 0.1% and 1%.

**Answer**

Convexity_adjustment = Convexity X 100 X (Dy)2

In [61]:
conv_adj = lambda C, Dy : C *100* Dy *2
Dy = 0.1 - 0.001

print("0.1% effect", conv_adj(conv,Dy))
Dy = 0.1 - 0.01
print("1% effect", conv_adj(conv,Dy))

0.1% effect 206.20326417074415
1% effect 187.4575128824947


- Compare your answer to the exact values.


If computing the exact value is so simple, why bothering using an approximation through
duration ?